# Necessary files

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**/!\ Check the path to the main directory**

Here I put the **IDL_group_project** directory at the root of the Google Drive

In [ ]:
# Check everytime
# cd to the main directory
%cd '/content/drive/MyDrive/IDL_group_project/Google Colab/'

data_dir = '/content/' # Google Colab
# data_dir = './' # Local Jupyter

/content/drive/.shortcut-targets-by-id/1iQGIRMVT6SqZTKFK67pUpzC6fBquoGX_/IDL_group_project/Google Colab


### PASCAL VOC2010

In [ ]:
# # cd to the correct PASCAL VOC2010 dataset location, which should contain tar files
# %cd 'data/PASCAL VOC2010/'
# !ls

# # Extract the data to /content/data/

# !mkdir '/content/data'
# !mkdir '/content/data/PASCAL VOC2010'

# !mkdir '/content/data/PASCAL VOC2010/VOCtrainval_03-May-2010' # Train & val datasets
# !tar -xf VOCtrainval_03-May-2010.tar -C '/content/data/PASCAL VOC2010/VOCtrainval_03-May-2010' --skip-old-files # Train & val datasets

# # !mkdir '/content/PASCAL VOC2010/download' # Test dataset
# # !tar -xf download.tar -C '/content/data/PASCAL VOC2010/download' --skip-old-files # Test dataset

# # !mkdir '/content/PASCAL VOC2010/VOCdevkit_08-May-2010' # Optional
# # !tar -xf VOCdevkit_08-May-2010.tar -C '/content/data/PASCAL VOC2010/VOCdevkit_08-May-2010' --skip-old-files # Optional

# # cd back to the main working directory, all paths are referenced to this directory
# %cd ../../
# !ls

### PASCAL-Part

In [ ]:
# cd to the correct PASCAL-Part dataset location, which should contain tar.gz file
%cd 'data/PASCAL-Part/'
!ls

# Extract the data to /content/data/

!mkdir '/content/data'
!mkdir '/content/data/PASCAL-Part'

!mkdir '/content/data/PASCAL-Part/trainval'
!tar -xzf trainval.tar.gz -C '/content/data/PASCAL-Part/trainval' --skip-old-files

# cd back to the main working directory, all paths are referenced to this directory
%cd ../../
!ls

/content/drive/.shortcut-targets-by-id/1iQGIRMVT6SqZTKFK67pUpzC6fBquoGX_/IDL_group_project/Google Colab/data/PASCAL-Part
trainval.tar.gz
mkdir: cannot create directory ‘/content/data’: File exists
mkdir: cannot create directory ‘/content/data/PASCAL-Part’: File exists
mkdir: cannot create directory ‘/content/data/PASCAL-Part/trainval’: File exists
/content/drive/.shortcut-targets-by-id/1iQGIRMVT6SqZTKFK67pUpzC6fBquoGX_/IDL_group_project/Google Colab
 data				 PASCAL-Part_sample_usage.ipynb
'Detecting result.gsheet'	 save
 logs				 src
 models				 VGG16_detector_baseline.ipynb
 PASCAL-Part-pkl_creator.ipynb


# Import

In [ ]:
import os
import sys
import time
import csv

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import transforms
import torchvision.models as models

# !pip install torchinfo
# from torchinfo import summary

import numpy as np

from PIL import Image

import matplotlib.pyplot as plt

# !pip install adversarial-robustness-toolbox
# from art.estimators.classification import PyTorchClassifier
# from art.attacks.evasion import FastGradientMethod

sys.path.append('src')
from art_attack import test_art
from dataset import normalize, inverse_normalize, create_file_list, create_file_and_label_list, VOCImageFolder, visualize_images
from model import VOCDetector, train_detector, test_detector, write_csv_header, write_csv_params
from utilities import read_PASCAL_Part_to_dict

In [ ]:
print('Python version  ' + sys.version)
print('PyTorch version ' + torch.__version__)
print('Numpy version   ' + np.__version__)

Python version  3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]
PyTorch version 1.10.0+cu111
Numpy version   1.19.5


# Config

In [ ]:
category = 'car' # <---
categories = ['aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train']

!mkdir 'data/PASCAL-Part-anno-pkl'
!mkdir 'data/PASCAL-Part-anno-pkl/aeroplane'
!mkdir 'data/PASCAL-Part-anno-pkl/bicycle'
!mkdir 'data/PASCAL-Part-anno-pkl/boat'
!mkdir 'data/PASCAL-Part-anno-pkl/bus'
!mkdir 'data/PASCAL-Part-anno-pkl/car'
!mkdir 'data/PASCAL-Part-anno-pkl/motorbike'
!mkdir 'data/PASCAL-Part-anno-pkl/train'

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Using cuda device
Tesla K80


# Dataset

### PASCAL VOC2010

Lists of filenames and labels

In [ ]:
PASCAL_label_dir = data_dir + 'data/PASCAL VOC2010/VOCtrainval_03-May-2010/VOCdevkit/VOC2010/ImageSets/Main/'
PASCAL_image_dir = data_dir + 'data/PASCAL VOC2010/VOCtrainval_03-May-2010/VOCdevkit/VOC2010/JPEGImages/'

print(PASCAL_label_dir)
print(PASCAL_image_dir)

/content/data/PASCAL VOC2010/VOCtrainval_03-May-2010/VOCdevkit/VOC2010/ImageSets/Main/
/content/data/PASCAL VOC2010/VOCtrainval_03-May-2010/VOCdevkit/VOC2010/JPEGImages/


### PASCAL-Part

In [ ]:
PASCAL_Part_dir = data_dir + 'data/PASCAL-Part/trainval/Annotations_Part/'

PASCAL_Part_file_name_list, PASCAL_Part_file_path_list = create_file_list(PASCAL_Part_dir)

print(PASCAL_Part_file_name_list[:5])
print(PASCAL_Part_file_path_list[:5])

['2008_000002.mat', '2008_000003.mat', '2008_000007.mat', '2008_000008.mat', '2008_000009.mat']
['/content/data/PASCAL-Part/trainval/Annotations_Part/2008_000002.mat', '/content/data/PASCAL-Part/trainval/Annotations_Part/2008_000003.mat', '/content/data/PASCAL-Part/trainval/Annotations_Part/2008_000007.mat', '/content/data/PASCAL-Part/trainval/Annotations_Part/2008_000008.mat', '/content/data/PASCAL-Part/trainval/Annotations_Part/2008_000009.mat']


In [ ]:
index = PASCAL_Part_file_name_list.index('2008_000075.mat')
image_anno_dict = read_PASCAL_Part_to_dict(PASCAL_Part_file_path_list[index])

print(image_anno_dict['image_name'])
for i in range(image_anno_dict['num_objects']):
    obj = image_anno_dict['objects_list'][i]
    print(f'Object #{i}:')
    print('\tClass: ' + obj['obj_class'])
    print('\tMask: ' + str(obj['obj_mask'].shape))
    for j in range(obj['num_parts']):
        part = obj['parts_list'][j]
        print(f'\tPart #{j}:')
        print('\t\tPart name: ' + part['part_name'])
        print('\t\tMask: ' + str(part['part_mask'].shape))

2008_000075
Object #0:
	Class: bus
	Mask: (375, 500)
Object #1:
	Class: bus
	Mask: (375, 500)
	Part #0:
		Part name: frontside
		Mask: (375, 500)
	Part #1:
		Part name: rightside
		Mask: (375, 500)
	Part #2:
		Part name: door_1
		Mask: (375, 500)
	Part #3:
		Part name: rightmirror
		Mask: (375, 500)
	Part #4:
		Part name: headlight_1
		Mask: (375, 500)
	Part #5:
		Part name: wheel_1
		Mask: (375, 500)
	Part #6:
		Part name: wheel_2
		Mask: (375, 500)
	Part #7:
		Part name: window_1
		Mask: (375, 500)
	Part #8:
		Part name: window_2
		Mask: (375, 500)
	Part #9:
		Part name: window_3
		Mask: (375, 500)


In [ ]:
import pickle

save_dir = 'data/PASCAL-Part-anno-pkl/'
count = 0
total_file = len(PASCAL_Part_file_path_list)
for file_path in PASCAL_Part_file_path_list:
    image_anno_dict = read_PASCAL_Part_to_dict(file_path)
    for i in range(image_anno_dict['num_objects']):
        obj = image_anno_dict['objects_list'][i]
        category = obj['obj_class']
        if category in categories:
            save_path = save_dir + category + '/' + image_anno_dict['image_name'] + '.pkl'
            pkl_file = open(save_path, 'wb')
            pickle.dump(image_anno_dict, pkl_file)
            pkl_file.close()
    count += 1
    if count%100 == 0:
        print(f'Processed {count}/{total_file} files!')

Processed 100/10103 files!
Processed 200/10103 files!
Processed 300/10103 files!
Processed 400/10103 files!
Processed 500/10103 files!
Processed 600/10103 files!
Processed 700/10103 files!
Processed 800/10103 files!
Processed 900/10103 files!
Processed 1000/10103 files!
Processed 1100/10103 files!
Processed 1200/10103 files!
Processed 1300/10103 files!
Processed 1400/10103 files!
Processed 1500/10103 files!
Processed 1600/10103 files!
Processed 1700/10103 files!
Processed 1800/10103 files!
Processed 1900/10103 files!
Processed 2000/10103 files!
Processed 2100/10103 files!
Processed 2200/10103 files!
Processed 2300/10103 files!
Processed 2400/10103 files!
Processed 2500/10103 files!
Processed 2600/10103 files!
Processed 2700/10103 files!
Processed 2800/10103 files!
Processed 2900/10103 files!
Processed 3000/10103 files!
Processed 3100/10103 files!
Processed 3200/10103 files!
Processed 3300/10103 files!
Processed 3400/10103 files!
Processed 3500/10103 files!
Processed 3600/10103 files!
P

In [ ]:
pkl_file = open(save_path, 'rb')
image_sample = pickle.load(pkl_file)

print(image_sample['image_name'])
for i in range(image_sample['num_objects']):
    obj = image_sample['objects_list'][i]
    print(f'Object #{i}:')
    print('\tClass: ' + obj['obj_class'])
    print('\tMask: ' + str(obj['obj_mask'].shape))
    for j in range(obj['num_parts']):
        part = obj['parts_list'][j]
        print(f'\tPart #{j}:')
        print('\t\tPart name: ' + part['part_name'])
        print('\t\tMask: ' + str(part['part_mask'].shape))

2010_006082
Object #0:
	Class: aeroplane
	Mask: (332, 500)
	Part #0:
		Part name: body
		Mask: (332, 500)
	Part #1:
		Part name: lwing
		Mask: (332, 500)
	Part #2:
		Part name: rwing
		Mask: (332, 500)
	Part #3:
		Part name: engine_1
		Mask: (332, 500)
	Part #4:
		Part name: engine_2
		Mask: (332, 500)
	Part #5:
		Part name: stern
		Mask: (332, 500)
